<a href="https://colab.research.google.com/github/A01793146/iayaa-Equipo-48/blob/main/MNA_IAyAA_semana_9_Actividad_RedesNeuronales_Equipo48.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Maestría en Inteligencia Artificial Aplicada**
## **Curso: Inteligencia Artificial y Aprendizaje Automático**
### Tecnológico de Monterrey
### Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 7**
### **Red Neuronal Artificial - Perceptrón Multicapa : Multilayer Perceptrón (MLP)**
### Equipo 48

**Nombres y matrículas de los integrantes del equipo:**

* Axel Alejandro Tlatoa Villavicencio - A01363351
* Christopher Valdéz Cantú - A01793549
* Ernesto Nicanor Santillán Guerrero - A01793675
* Alejandro Jesús Vázquez Navarro - A01793146  



En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [ ]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import pandas as pd
import numpy as np
import sklearn 
from sklearn.model_selection import train_test_split 

# Toda la navaja suiza para machine learning
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import KFold

# Modelos
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor


# Para validación cruzada
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedKFold

#Para probar las métricas
from sklearn.metrics import classification_report, make_scorer
from sklearn import metrics 
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score


# Para tabulación
from tabulate import tabulate

# Para las curvas de aprendizaje
from sklearn.model_selection import learning_curve, validation_curve

# Para el Scaling
from sklearn.preprocessing import StandardScaler   

# Para el fine tuning
from sklearn.model_selection import GridSearchCV


# Para graficar
import matplotlib.pyplot as plt 
import seaborn as sns 

# Para transformación
from sklearn.preprocessing import power_transform

# Eliminar las warnings
import warnings
warnings.filterwarnings("ignore")

# **Ejercicio-1.** 


1- En esta tarea considera únicamente la siguiente variable de salida que se concluye que es una de las 
mejores en el artículo antes citado: ‘Lifetime People who have liked a Page and engaged with a 
post'. 

Renombra dicha variable como “LPE” . Como variables de entrada selecciona las 7 variables 
que indican los autores en la Tabla 3 del artículo citado. 

In [ ]:
#Lectura de los datos
dfMain = pd.read_csv("dataset_Facebook.csv", sep=";", index_col=False)

#Renombrar variable solicitada
dfMain.rename(columns ={'Lifetime People who have liked your Page and engaged with your post':'LPE'}, inplace=True)

dfMain = dfMain[["LPE", "Category", "Page total likes", "Post Month", "Post Hour", "Post Weekday", "Paid"]]

print(f'Total de registros en dfMain: {len(dfMain)}')
#Selección de las variables predictoras de acuerdo al artículo
X = dfMain.drop('LPE', axis=1)

print(f'Columnas de  {X.columns}')

Y = dfMain["LPE"]



# **Ejercicio-2.**

2- Realiza una partición de los datos con 100 datos de Prueba y el resto para entrenamiento y 
validación

In [ ]:
# Utilizamos train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.80, random_state=10)

print(f'Tamaño de X prueba: {X_test.shape}')
print(f'Tamaño de Y prueba: {y_test.shape}')
 
print(f'Tamaño de X entrenamiento: {X_train.shape}')
print(f'Tamaño de Y entrenamiento: {y_train.shape}')
      

# **Ejercicio-3.**

3- Definirás tus propias funciones de errores para este problema de regresión. 
Los errores que utilizarás son la **raíz cuadrada del error cuadrático medio RMSE**, **el error absoluto medio MAE** y el **error porcentual absoluto medio MAPE.** 

In [ ]:
# Definición de funciones de errores:

def mi_RMSE(y_val_, yhatVal):
    mse =  np.square(np.subtract(y_val_, yhatVal)).mean()
    rmse = np.sqrt(mse)
    
    return rmse


# Incluye aquí las líneas de código que definan a la función: mi_MAE:
def mi_MAE(y_val_, yhatVal):

    y_val_ = np.array(y_val_)
    yhatVal = np.array(yhatVal)

    return np.mean(np.abs(y_val_ - yhatVal))


# Incluye aquí las líneas de código que definan a la función: mi_MAPE:
def mi_MAPE(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [ ]:
dfMain.isna().sum().sum()
dfMain.isnull()
# Tenemos un valor nulo

# **Ejercicio-4.**

4- En la página de la UCI, así como en el artículo de los autores previamente citado encuentras 
información en relación al significado de cada variable. 

* Haz una análisis de tus datos y lleva a cabo las transformaciones que consideres adecuadas tanto en los datos de entrada, como en las de salida. 

* Utiliza un Pipeline para evitar el filtrado de información. 


In [ ]:
# Analicemos si existen datos perdidos



# Lo primero que debemos hacer es identificar los tipos de variable, posteriormente identificar dónde hay valors NaN, 
# calcular distribuciones y posteriormente determinar el mejor criterio para imputar datos faltantes
# Identificación de tipo de variables

#Variables numéricas
#Removí 'Category' y 'Paid' porque son variables nominales

colCategoricas = ['Category', 'Paid']
colNumericas =['Page total likes', 'Post Month', 'Post Hour','Post Weekday']

#Todas son variables numéricas
#Veamos cómo están sus distribuciones y sus escalas

sns.set(rc={'figure.figsize':(17,18)})
fig, axes = plt.subplots(4, 5)    
  
for k in range(0,4):       
 
  
    # Datos originales ---------------------------------------------------------
    plt.subplot(5,4,k+1) 

    Transf0 = dfMain[colNumericas[k]]         
    sns.histplot(data=Transf0, bins=20)                    

    plt.xlabel(colNumericas[k])
    if k==0:
        plt.ylabel('Originales')
    


    # Datos transformados con raíz cuadrada ------------------------------------
    plt.subplot(5,4,k+5)    

    Transf1 = np.sqrt(dfMain[colNumericas[k]]) 
    sns.histplot(data=Transf1, bins=20)                     


    plt.xlabel(colNumericas[k])
    if k==0:
        plt.ylabel('Raíz Cuadrada')
    

    
    # Datos transformados con logaritmo natural --------------------------------
    plt.subplot(5,4,k+9)     

    Transf2 = np.log(dfMain[colNumericas[k]])   
    sns.histplot(data=Transf2, bins=20)                     


    plt.xlabel(colNumericas[k])
    if k==0:
        plt.ylabel('Logaritmo')
    


    # Datos transformados con la potencia de 2 ---------------------------------
    plt.subplot(5,4,k+13)

    Transf3 = np.power(dfMain[colNumericas[k]], 2)
    sns.histplot(data=Transf3, bins=20)                        


    plt.xlabel(colNumericas[k])
    if k==0:
        plt.ylabel('Potencia 2')
    
    # Datos transformados con Box-Cox ------------------------------------------
    plt.subplot(5,4,k+17)

    Transf4 = power_transform(np.array(dfMain[colNumericas[k]]).reshape(-1, 1), method="box-cox")  
    sns.histplot(Transf4, bins=20)                               


    plt.xlabel(colNumericas[k])
    if k==0:
        plt.ylabel('Box-Cox')

    

plt.show()

In [ ]:
# De acuerdo a este análisis, será conveniente aplicar transformaciones de MinMaxScaler y SimpleImputer para 
# las variables con NaN

num_pipeline = Pipeline(steps = [('impMediana', SimpleImputer(strategy='median')),
                                 ('escalaNum', MinMaxScaler(feature_range=(1,2)))])   

cat_pipeline = Pipeline(steps = [('impMediana', SimpleImputer(strategy='median'))])  

num_pipeline_nombres = colNumericas
cat_pipeline_nombres = colCategoricas

# Transformación One-Hot encoding a variables de entrada de tipo categórico:
# catOHE_pipeline = Pipeline(steps = [('OneHotE', OneHotEncoder(drop='first', handle_unknown='ignore'))])
# catOHE_pipeline_nombres = colCategoricas


# Creación de Pipeline:
# columnasTransformer = ColumnTransformer(transformers = [('numpipe', num_pipeline, num_pipeline_nombres),
#                                                        ('catohe', catOHE_pipeline, catOHE_pipeline_nombres)],
#                                                        remainder='passthrough')


# Creación de Pipeline:
columnasTransformer = ColumnTransformer(transformers = [
                                    ('numpipe', num_pipeline, num_pipeline_nombres),
                                    ('catpipe', cat_pipeline, cat_pipeline_nombres)
                                                       ],
                               remainder='passthrough')

# **Ejercicio-5.**

5- Utiliza la función Dummy para modelos de regresión de scikit-learn con el conjunto que tienes de 
datos de entrenamiento y validación. 

- Para ello particiónalos en 100 para validación y 300 para entrenamiento. Encuentra los errores RMSE, MAE y MAPE para los conjuntos de entrenamiento y validación. 

- Estos serán tus errores máximos que deberás tomar como referencia en el resto de la 
actividad. 

- Consulta su documentación correspondiente: 
https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html 



In [ ]:
#Particionar los datos de entrenamiento en 100 para validación y 300 para entrenamiento
X__train, X__val, y__train, y__val = train_test_split(X_train, y_train, test_size=100/400, random_state=10)

#Verificamos el tamaño

print(f'Tamaño de las X de Validación X__val {X__val.shape}')
print(f'Tamaño de las Y de Validación y__val {y__val.shape}')
print('---' * 20)
print(f'Tamaño de las X de Training X__train {X__train.shape}')
print(f'Tamaño de las Y de Training Y__train {X__train.shape}')



In [ ]:

modelo_dummy= DummyRegressor().fit(X__train, y__train)

y_pred = modelo_dummy.predict(X__val)
y_pred_train = modelo_dummy.predict(X__train)

print("Mi baseline Validación:")
print("-" * 50)
print('Mi RMSE:', mi_RMSE(y__val, y_pred))
print('Mi MAE:', mi_MAE(y__val, y_pred))
print('Mi MAPE:', mi_MAPE(y__val, y_pred))


print("-" * 50)
print("Mi baseline Entrenamiento:")
print("-" * 50)
print('Mi RMSE:', mi_RMSE(y__train, y_pred_train))
print('Mi MAE:', mi_MAE(y__train, y_pred_train))
print('Mi MAPE:', mi_MAPE(y__train, y_pred_train))


# **Ejercicio-6.**

6- Usando los modelos de regresión lineal múltiple, el bosque aleatorio y el perceptrón multicapa con 
sus valores predeterminados, lleva a cabo su entrenamiento con repeticiones de validación cruzada 
(RepeatedKFold) y desplegando los errores RMSE, MAE y MAPE. 

Recuerda evitar el filtrado de información usando los datos que obtuviste en el ejercicio 2. Incluye las conclusiones sobre el mejor modelo encontrado en esta primera aproximación. En particular ¿hay alguno sobreentrenado o 
subentrenado? 

**NOTA: Recuerda que puedes aumentar en dado caso el número máximo de iteraciones para que todos los modelos converjan.**

In [ ]:
def get_modelos():
    
    modelos, nombres = list(), list()

    modelos.append(LinearRegression())
    nombres.append('LinearRegression')
    
    modelos.append(RandomForestRegressor())
    nombres.append('RandomForest')
    
    modelos.append(MLPRegressor(solver='lbfgs', 
                              alpha=1e-5,
                              hidden_layer_sizes=(6,100), 
                              random_state=1))
                       
                  
    nombres.append('Perceptrón Multicapa')
    
    
    
    return modelos, nombres


In [ ]:
modelosOU, nombres = get_modelos()       
resultados = list()
resultadosTodos = list()

for i in range(len(modelosOU)):

    cv = RepeatedKFold(n_splits=5, n_repeats=20, random_state=123)
    pipe = Pipeline([('preprocessing', columnasTransformer),
                     ('modelo', modelosOU[i])])

    scoring = {'RSME': make_scorer(mi_RMSE), 'MAE': make_scorer(mi_MAE), 'MAPE': make_scorer(mi_MAPE)}

    cv_scores = cross_validate(
                    estimator = pipe,
                    X         = X,
                    y         = Y,
                    scoring   = scoring,
                    cv        = cv,
                    return_train_score = True
                 )

    resultadosTodos.append(cv_scores)
    
    train_mae  = round(cv_scores['train_MAE'].mean(),5)
    train_rsme = round(cv_scores['train_RSME'].mean(), 5)
    train_mape = round(cv_scores['train_MAPE'].mean(), 5)

    test_mae  = round(cv_scores['test_MAE'].mean(),5)
    test_rsme = round(cv_scores['test_RSME'].mean(), 5)
    test_mape = round(cv_scores['test_MAPE'].mean(), 5)
    resultados.append([train_mae, test_mae, train_rsme, test_rsme, train_mape, test_mape])
# Se convierte el diccionario a dataframe para facilitar la visualización


dfScores = pd.DataFrame(resultados, columns = ['Train MAE', 'Test MAE', 'Train RSME', 'Test RSME', 'Train MAPE', 'Test MAPE'], 
             index=nombres).sort_values('Train MAE', ascending=False)

dfScores


De acuerdo a estos resultados, podemos observar:
- La regresión linear tiene baja varianza (Train MAPE (137.69) - Test MAPE (140.26) pero alto bias (137.69 en el entrenamiento; este valor está muy alejado de 0 (cero)), esto a todas luces es un modelo **subentrenado** lo cual es lógico debido a que no tien
e afinamiento de hiperparámetros.

- El modelo de perceptrón multicapa tiene baja varianza (Train MAPE (130.15) - Test MAPE (134.29) pero alto bias (120.15 en el entrenamiento; este valor está muy alejado de 0 (cero)), esto a todas luces es un modelo **subentrenado** lo cual es lógico debido a que no tiene afinamiento de hiperparámetros.

- El modelo de Random Forest tiene alta varianza (Train 54.2) - Test MAPE (140.4) y un relativo bajo sesgo (54.22 en el entrenamiento; este valor no está tan alejado de cero), esto puede interpretarse como un modelo ** sobreentrenado **, es natural porque no tiene afinación de hiperparámetros.

#**Ejercicio-7.**

7- Obtener los diagramas de caja y bigote para los errores MAPE de los conjuntos de validación 
obtenidos. 

En particular compara estos primeros resultados de MAPE con el mejor resultado que 
encuentran los autores del artículo citado al inicio. Incluye tus conclusiones.

In [ ]:
sns.set(rc={'figure.figsize':(8,4)})



MAPE = list()
for i in range(len(resultadosTodos)):
  rr = resultadosTodos[i]['test_MAPE']
  MAPE.append(rr)

plt.boxplot(MAPE, labels=nombres, showmeans=True)


plt.show()

Comparando estos resultados contra los obtenidos por los autores del 2016 de Moro, Rita & Vala (https://www.semanticscholar.org/paper/Predicting-social-media-performance-metrics-and-of-Moro-Rita/dec55692590820754b53c916e29bb2b42c0e5104) los modelos desempeñan pobremente. 

La métrica de comparación es "Lifetime people who have liked you page and engaged with your post" y obtuvo en el artículo un MAPE de 26.5; nuestro mejor modelo (hasta este momento es el Random Forest) con un MAPE de 54.27 promedio. Esto indica que debemos hacer ajustes aún a nuestros modelos con un gridsearch. Habría sido sorprendente que en esta primera iteración obtuviéramos métricas de performance alto.


#**Ejercicio-8.**

- Usando una búsqueda de malla con validación cruzada (GridSearchCV), busca los mejores 
hiperparámetros para el modelo MLP. Al menos deberás realizar la búsqueda en los 
hiperparámetros “hidden_layer_sizes”, “alpha” y “learning_rate_init”. Además aplica la validación 
cruzada con repeticiones (RepeatedKFold). Muestra los mejores hiperparámetros encontrados. 


In [ ]:
modelo = MLPRegressor(hidden_layer_sizes=(6,100), 
                      solver='adam',               
                      random_state=1)

        
dicc_grid = {"hidden_layer_sizes": [1,50], "alpha": [0.00005,0.0005], "learning_rate_init": np.arange(0.01,1.01,0.1) }

scoring = {'MAPE': make_scorer(mi_MAPE)}

cv = RepeatedKFold(n_splits=5, n_repeats=20, random_state=123)

grid = GridSearchCV(estimator=modelo, 
                    param_grid=dicc_grid, 
                    cv=cv,    
                    verbose=5,
                    n_jobs=-1,
                    scoring   = 'neg_mean_absolute_percentage_error'
                    )

# Transformamos los datos de entrada:
Xx = columnasTransformer.fit_transform(X_train)

grid.fit(Xx, np.ravel(y_train))

print('Mejor valor de exactitud obtenido con la mejor combinación:', grid.best_score_)
print('Mejor combinación de valores encontrados de los hiperparámetros:', grid.best_params_)
print('Métrica utilizada:', grid.scoring)

#**Ejercicio-9.**

Con los mejores valores de los hiperparámetros encontrados realiza un análisis de la importancia de 
los factores. Muestra un diagrama de barras de los resultados e incluye tus conclusiones. 

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
# Una vez inicializado y entrenada la MLP, veamos qué factores considera más importantes:

modelo_MLP = MLPRegressor(
                            alpha=5e-05,
                            hidden_layer_sizes= 1,
                            learning_rate_init=0.01)  

# Transformamos los datos de entrada:
Xx = columnasTransformer.fit_transform(X_train)

modelo_MLP.fit(Xx,  np.ravel(y_train))

importance = permutation_importance(modelo_MLP, Xx, np.ravel(y_train), n_repeats=10)

# visualicemos la importancia de cada métrica, de las cuales sabemos
# cuáles son las más importantes, de acuerdo a como se definieron al inicio:

for i,v in enumerate(importance['importances_mean']):
	print('Feature: %0d, Score: %.9f' % (i,v))
 
plt.bar([x for x in range(len(importance['importances_mean']))], importance['importances_mean'])
plt.show()

#**Ejercicio-10.**

Repite el ejercicio 8 y 9 para el modelo de **bosque aleatorio** para buscar sus mejores 
hiperparámetros (realiza la búsqueda con aquellos hiperparámetros que consideres más 
adecuados) y usando el conjunto de Prueba. Y realiza igualmente el análisis de importancia de 
factores con este modelo con un diagrama de barras

In [ ]:
# Bosque Aleatorio
modelo = RandomForestRegressor()
        
dicc_grid = {
    'n_estimators': [100, 150, 200, 250, 300],
    'max_depth': [1,2,3,4],
}

cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=8)

grid = GridSearchCV(estimator=modelo, 
                    param_grid=dicc_grid, 
                    cv=cv, 
                    verbose=5,
                    n_jobs=-1,
                    scoring   = 'neg_mean_absolute_percentage_error'
                    )

# Transformamos los datos de entrada:
Xx = columnasTransformer.fit_transform(X_test)

grid.fit(Xx, np.ravel(y_test))

print('Mejor valor de exactitud obtenido con la mejor combinación:', grid.best_score_)
print('Mejor combinación de valores encontrados de los hiperparámetros:', grid.best_params_)
print('Métrica utilizada:', grid.scoring)

In [ ]:
# Importancias de las características del RandomForestRegressor

modelo_RF = RandomForestRegressor(
                           max_depth=2,
                           n_estimators = 150
                           )  

# Transformamos los datos de entrada:
Xx = columnasTransformer.fit_transform(X_test)

modelo_RF.fit(Xx,  np.ravel(y_test))

importance = permutation_importance(modelo_RF, Xx, np.ravel(y_test), n_repeats=10)

# visualicemos la importancia de cada métrica, de las cuales sabemos
# cuáles son las más importantes, de acuerdo a como se definieron al inicio:

for i,v in enumerate(importance['importances_mean']):
	print('Feature: %0d, Score: %.9f' % (i,v))
 
plt.bar([x for x in range(len(importance['importances_mean']))], importance['importances_mean'])
plt.show()

#**Ejercicio-11.**

Repite el ejercicio 8 y 9 para el modelo de **regresión lineal múltiple** para buscar sus mejores 
hiperparámetros (realiza la búsqueda con aquellos hiperparámetros que consideres más 
adecuados) y usando el conjunto de Prueba. Y realiza igualmente el análisis de importancia de 
factores con este modelo con un diagrama de barras

In [ ]:
#Usando Ridge

# Regresión Lineal Múltiple
modelo = LinearRegression()

dicc_grid={'fit_intercept':[True, False], 'normalize': [True, False], 'copy_X':[True, False]}

cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=8)

grid = GridSearchCV(estimator=modelo, 
                    param_grid=dicc_grid, 
                    cv=cv, 
                    verbose=5,
                    n_jobs=-1,
                    scoring   = 'neg_mean_absolute_percentage_error'
                    )

# Transformamos los datos de entrada:
Xx = columnasTransformer.fit_transform(X_test)

grid.fit(Xx, np.ravel(y_test))

print('Mejor valor de exactitud obtenido con la mejor combinación:', grid.best_score_)
print('Mejor combinación de valores encontrados de los hiperparámetros:', grid.best_params_)
print('Métrica utilizada:', grid.scoring)
print("---" * 50)


In [ ]:
# Importancia de las características de Regresión Lineal Múltiple

modelo_RL = LinearRegression(
                           copy_X=True,
                           fit_intercept = True,
                           normalize = True
                           )  

# Transformamos los datos de entrada:
Xx = columnasTransformer.fit_transform(X_test)

modelo_RL.fit(Xx,  np.ravel(y_test))

importance = permutation_importance(modelo_RL, Xx, np.ravel(y_test), n_repeats=10)

# visualicemos la importancia de cada métrica, de las cuales sabemos
# cuáles son las más importantes, de acuerdo a como se definieron al inicio:

for i,v in enumerate(importance['importances_mean']):
	print('Feature: %0d, Score: %.9f' % (i,v))
 
plt.bar([x for x in range(len(importance['importances_mean']))], importance['importances_mean'])
plt.show()

#**Ejercicio-12.**

- Compara tus resultados con los obtenidos por los autores del artículo de Moro-Rita-Vala con 
respecto a MAPE. Incluye tus conclusiones finales de la actividad.

In [ ]:
None

###**Fin de la Actividad de la semana 7.**